# Misson 3

## 기능요구사항
- IMF에서 제공하는 국가별 GDP를 구하세요.

- 국가별 GDP를 확인할 수 있는 테이블을 만드세요.

- 해당 테이블에는 GDP가 **높은 국가들이 먼저** 나와야 합니다.

- GDP의 단위는 1B USD이어야 하고 **소수점 2자리**까지만 표시해 주세요.

- IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 **해당 코드를 재사용**해서 정보를 얻을 수 있어야 합니다.

## 프로그래밍 요구 사항
- **주석**을 사용해서 설명을 추가하고 
  
- 함수를 만들어서 가독성과 재사용성을 높이세요.
  
- 추출 한 정보는 '**Countries_by_GDP.json**'라는 이름의 JSON 화일 포맷으로 저장해야 합니다.
  
- 필요한 모든 작업을 수행하는 '**etl_project_gdp.py**' 코드를 작성하세요.

## 화면 출력
- **GDP가 100B USD이상**이 되는 국가만을 구해서 화면에 출력해야 합니다.

- 각 **Region별로 top5** 국가의 **GDP 평균**을 구해서 화면에 출력해야 합니다.

## ETL 프로세스
- **ETL 프로세스**에 따라 코드를 작성하고 각 단계의 **시작과 끝을 로그**에 기록하세요.
- 

- 이 모든 처리 과정은 'etl_project_log.txt'라는 로그 화일에 기록되어야 합니다. (로그 화일은 매번 다시 생성하는 것이 아니라 **기존 화일에 append**되어야 합니다.)

- log는 "**time, log**" 형식으로 기록하세요. 시간은 'Year-Monthname-Day-Hour-Minute-Second' 포맷에 맞게 표시하세요.

In [1]:
import requests
from bs4 import BeautifulSoup# 크롤링
import pandas as pd # 데이터 처리
import json # to json file
import datetime # logging
import sqlite3 # DB
import re
import time, datetime

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

In [3]:
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
else:
    print(response.status_code)
    

In [4]:
type(html)

str

In [5]:
''' 
Table Head Structure:
        | IMF               | WB                | UN                | 
Country |Forecast   | Year  |Forecast   | Year  |Forecast   | Year  |
        | :TODO     | :TODO |
----------------------------------------------------------------------       
Table Row Structure:
td      | td        | td    | td        | td    | td        | td    |
'''

' \nTable Head Structure:\n        | IMF               | WB                | UN                | \nCountry |Forecast   | Year  |Forecast   | Year  |Forecast   | Year  |\n        | :TODO     | :TODO |\n----------------------------------------------------------------------       \nTable Row Structure:\ntd      | td        | td    | td        | td    | td        | td    |\n'

In [34]:
df = pd.read_html(url, attrs={"class": "wikitable"})[0]

# 첫 번째 테이블을 가져옵니다 (여러 테이블이 있을 수 있으므로 인덱스로 선택)
# df = tables
# print(df)
print(type(df))
# df = pd.DataFrame(df)
print(df)
# MultiIndex 생성
columns = [
    ["Country"] + ["IMF"] * 2 + ["WB"] * 2 + ["UN"] * 2,
    [""] + ["Forecast", "Year"] * 3  # 하위 컬럼: Forecast와 Year 반복
]

# MultiIndex로 설정
df.columns = pd.MultiIndex.from_arrays(columns)

# 결과 확인
df.iloc[-24]

<class 'pandas.core.frame.DataFrame'>
    Country/Territory IMF[1][13]            World Bank[14]             \
    Country/Territory   Forecast       Year       Estimate       Year   
0               World  115494312       2025      105435540       2023   
1       United States   30337162       2025       27360935       2023   
2               China   19534894  [n 1]2025       17794782  [n 3]2023   
3             Germany    4921563       2025        4456081       2023   
4               Japan    4389326       2025        4212945       2023   
..                ...        ...        ...            ...        ...   
205          Kiribati        311       2024            279       2023   
206             Palau        308       2024            263       2023   
207  Marshall Islands        305       2024            284       2023   
208             Nauru        161       2024            154       2023   
209            Tuvalu         66       2024             62       2023   

    United N

Country              Zanzibar
IMF      Forecast           —
         Year               —
WB       Forecast           —
         Year               —
UN       Forecast        2361
         Year            2022
Name: 186, dtype: object

In [49]:
import os
current_dir = os.path.dirname(os.getcwd())
print(current_dir)
raw_json = 'Countries_by_GDP.json'
with open(raw_json, "r") as raw_Data:
    json_raw = json.load(raw_Data)


/Users/admin/Documents/GitHub/Softeer-Bootcamp/missions


In [59]:
og_json = pd.read_json(raw_json)
columns = [
    ["Country"] + ["IMF"] * 2 + ["WB"] * 2 + ["UN"] * 2,
    [""] + ["Forecast", "Year"] * 3  # 하위 컬럼: Forecast와 Year 반복
]
og_json.columns = pd.MultiIndex.from_arrays(columns)
og_json['Country']
og_json = og_json[['Country', 'IMF']]
og_json['IMF', 'Forecast']

0      115494312
1       30337162
2       19534894
3        4921563
4        4389326
         ...    
205          311
206          308
207          305
208          161
209           66
Name: (IMF, Forecast), Length: 210, dtype: object

In [73]:
table = soup.select('table.wikitable > tbody > tr:nth-child(n+4)')#.wikitable sortable sticky-header-multi static-row-numbers jquery-tablesorter')
# tr, td 등의 tag가 변수 취급이 가능하구나
gdp_rows = [[td.get_text(strip=True) for td in tr.find_all('td')] for tr in table]

print(gdp_rows)

[['United States', '30,337,162', '2025', '27,360,935', '2023', '25,744,100', '2022'], ['China', '19,534,894', '[n 1]2025', '17,794,782', '[n 3]2023', '17,963,170', '[n 1]2022'], ['Germany', '4,921,563', '2025', '4,456,081', '2023', '4,076,923', '2022'], ['Japan', '4,389,326', '2025', '4,212,945', '2023', '4,232,173', '2022'], ['India', '4,271,922', '2025', '3,549,919', '2023', '3,465,541', '2022'], ['United Kingdom', '3,730,261', '2025', '3,340,032', '2023', '3,089,072', '2022'], ['France', '3,283,429', '2025', '3,030,904', '2023', '2,775,316', '2022'], ['Italy', '2,459,597', '2025', '2,254,851', '2023', '2,046,952', '2022'], ['Canada', '2,330,308', '2025', '2,140,086', '2023', '2,137,939', '2022'], ['Brazil', '2,307,162', '2025', '2,173,666', '2023', '1,920,095', '2022'], ['Russia', '2,195,708', '2025', '2,021,421', '2023', '2,240,422', '2022'], ['South Korea', '1,947,133', '2025', '1,712,793', '2023', '1,673,916', '2022'], ['Australia', '1,881,140', '2025', '1,723,827', '2023', '1,77

In [74]:
re.sub(r',', '', '10,000')

'10000'

In [ ]:
country = []
gdp = []
year = []
no_juseok = lambda s: re.sub(r'\[.*?\]', '', s)
str_to_num = lambda s: int(re.sub(r',', '', s))

for row in gdp_rows:
    print(row)
    country.append(no_juseok(row[0]))
    
    if row[1] == '—': gdp.append(-1)
    else: gdp.append(str_to_num(no_juseok(row[1])))
    
    if row[2] == '—': year.append(-1)
    else: year.append(no_juseok(row[2]))

In [98]:
print(len(country), len(gdp), len(year)) # checked!

209 209 209


In [ ]:
gdp = [round(i / 1000, 2) for i in gdp]
GDP_Table = pd.DataFrame({'Country': country, 'GDP': gdp, 'Year': year}).sort_values('GDP', ascending=False, inplace=True)

In [103]:
GDP_Table = GDP_Table[GDP_Table['GDP'] >= 100]

## Prototype

In [103]:
import requests
from bs4 import BeautifulSoup# 크롤링
import pandas as pd # 데이터 처리
import json # to json file
import datetime # logging
import sqlite3 # DB
import re
import time, datetime
import logging
from functools import wraps

class Extract:
    WIKI_URL = ''
    IMF_URL = ''
    
    def __init__(self):
        self.WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
        self.IMF_URL = 'https://www.imf.org/external/datamapper/api/v1/NGDPD'
    
    def url_validation_check(self, url:str):
        '''
        URL 유효성 검증
        Param: url: str
        return: html text
        '''
        response = requests.get(url)

        if response.status_code == 200:
            return response.text
        else:
            print(response.status_code)  
            return False      
    
    def collect_data(self, html: str):
        '''
        원본 데이터 수집
        Param: html: str
        return: gdp_rows: List 
        # 국가별 자료입니다. 형식: [[국가명, GDP, 연도, GDP, 연도, GDP, 연도] ... ] 
        '''
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.select('table.wikitable > tbody > tr:nth-child(n+4)')#.wikitable sortable sticky-header-multi static-row-numbers jquery-tablesorter')
        # tr, td 등의 tag가 변수 취급이 가능하구나
        gdp_rows = [[td.get_text(strip=True) for td in tr.find_all('td')] for tr in table]

        return gdp_rows

    def save_json(self, raws:list):
        '''
        데이터프레임을 JSON 형식으로 저장합니다.
        '''
        raw_json = {'country' : [], 
                    'forecast' : [], 
                    'year' : []
                    }
        for line in raws:
            raw_json['country'].append(line[0])
            
            if line[1] == '—':
                raw_json['forecast'].append('-')
                raw_json['year'].append('-')
            else:
                raw_json['forecast'].append(line[1])
                raw_json['year'].append(line[2])
                
        # try:
        filename = 'Countries_by_GDP.json'
        with open(filename, 'w') as f : 
            json.dump(raw_json, f, indent=4, ensure_ascii=False)
                     
        # except Exception as e:
        #     print(f"Error processing {filename}: {e}")

class Transform:
    raw_data = []
    def __init__(self, raw_data:list):
        '''
        데이터를 가공하기 위해 원본 데이터를 가져옵니다.
        '''
        self.raw_data = raw_data
    
    def refine_data(self):
        '''
        데이터를 정제합니다.
        링크를 지우고 문자열로 된 숫자를 정수로 변환합니다.
        '-'로 표기된 데이터는 -1 을 기입합니다.
        
        return: country: List, gdp: List, year: List
        '''
        country = []
        gdp = []
        # year = []
        
        no_juseok = lambda s: re.sub(r'\[.*?\]', '', s)
        str_to_num = lambda s: int(re.sub(r',', '', s))
        
        for i, row in enumerate(self.raw_data):

            if len(row) < 2: 
                print(f"Data Error in line {i}")
                
            country.append(no_juseok(row[0]))
            
            if row[1] == '—': 
                gdp.append(-1)
            else: 
                gdp.append(str_to_num(no_juseok(row[1])))
        
        return country, gdp
        
    def make_DataFrame(self, country:list, gdp:list):
        '''
        가공된 데이터로 데이터프레임을 생성합니다.
        GDP에 대해 내림차순 정렬되어 있고 GDP가 $100B 이상인 국가만 포함됩니다.

        Param: country: List, gdp: List, year: List
        return: GDP_data: pd.DataFrame
        '''
        GDP_data = pd.DataFrame({'Country': country, 'GDP_USD_Bilion': gdp})
        GDP_data.sort_values('GDP_USD_Bilion', ascending=False, inplace=True)
           
        GDP_data['GDP_USD_Bilion'] = GDP_data['GDP_USD_Bilion'].apply(lambda x: round(x / 1000, 2))
        
        
        return GDP_data[GDP_data['GDP_USD_Bilion'] >= 100]
    
class Load:
    frame = None
    def __init__(self, frame:pd.DataFrame):
        '''
        데이터프레임을 저장하기 위해 불러옵니다.
        '''
        self.frame = frame
    
            
    def save_db(self,):
        '''
        데이터프레임을 DB에 저장합니다.
        '''
        try:
            with sqlite3.connect('World_Economies.db') as conn:
                print(f"Opened SQLite database with version {sqlite3.sqlite_version} successfully.")

        except sqlite3.OperationalError as e:
            print("Failed to open database:", e)
            
        self.frame.to_sql('Countries_by_GDP', conn)
        
def visualize():
    # show top 5 countries of each Region
    REGION = {
        'Africa':[],
        'North Africa':[],
        'Sub-Saharan Africa':[],
        'Asia and Pacific':[],
        'Australia and New Zealand':[],
        'Central Asia and the Caucasus':[],
        'East Asia':[],
        'Pacific Islands':[],
        'South Asia':[],
        'Southeast Asia':[],
        'Europe':[],
        'Eastern Europe':[],
        'Western Europe':[],
        'Middle East':[],
        'Western Hemisphere':[],
        'Caribbean':[],
        'Central America':[],
        'North America':[],
        'South America':[]
    }

logger = logging.getLogger()
if not logger.handlers:
    # 사용자 정의 Formatter
    formatter = logging.Formatter(
        fmt='%(asctime)s, %(message)s',
        datefmt='%Y-%B-%d-%H-%M-%S'  # 원하는 날짜 포맷
    )

    # 핸들러 생성 및 설정
    handler = logging.FileHandler('etl_project_log.txt')
    handler.setFormatter(formatter)

    # 로거 설정
    logger.setLevel(logging.INFO)
    logger.addHandler(handler)

# 테스트 로그 메시지
logger.info("This is a test log message.")

def logging_time(func):
    '''
    @logging_time을 로깅하고 싶은 함수 윗줄에 작성한다.
    '''    
    @wraps(func)
    def wrapper(*args, **kwargs):
        # start_time = datetime.datetime.now()
        # formatted_start = start_time.strftime("%Y-%B-%d-%H-%M-%S")

        logging.info(f"Started '{func.__name__}'")
        
        result = func(*args, **kwargs)
        
        # end_time = datetime.datetime.now()
        # formatted_end = end_time.strftime("%Y-%B-%d-%H-%M-%S")

        logging.info(f"Finished '{func.__name__}'")
        # logging.info(f"Execution time for '{func.__name__}': {end_time - start_time}")
        return result
    return wrapper

## Test Code

In [104]:
e = Extract()
print(e.WIKI_URL)
if html := e.url_validation_check(e.WIKI_URL):
    raws = e.collect_data(html)
t = Transform(raws)
table = t.make_DataFrame(*t.refine_data())
table

https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29


,Country,GDP_USD_Bilion
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
68,Uzbekistan,112.65
69,Guatemala,112.37
70,Oman,109.99
71,Bulgaria,108.42


In [105]:
raws

[['United States',
  '30,337,162',
  '2025',
  '27,360,935',
  '2023',
  '25,744,100',
  '2022'],
 ['China',
  '19,534,894',
  '[n 1]2025',
  '17,794,782',
  '[n 3]2023',
  '17,963,170',
  '[n 1]2022'],
 ['Germany', '4,921,563', '2025', '4,456,081', '2023', '4,076,923', '2022'],
 ['Japan', '4,389,326', '2025', '4,212,945', '2023', '4,232,173', '2022'],
 ['India', '4,271,922', '2025', '3,549,919', '2023', '3,465,541', '2022'],
 ['United Kingdom',
  '3,730,261',
  '2025',
  '3,340,032',
  '2023',
  '3,089,072',
  '2022'],
 ['France', '3,283,429', '2025', '3,030,904', '2023', '2,775,316', '2022'],
 ['Italy', '2,459,597', '2025', '2,254,851', '2023', '2,046,952', '2022'],
 ['Canada', '2,330,308', '2025', '2,140,086', '2023', '2,137,939', '2022'],
 ['Brazil', '2,307,162', '2025', '2,173,666', '2023', '1,920,095', '2022'],
 ['Russia', '2,195,708', '2025', '2,021,421', '2023', '2,240,422', '2022'],
 ['South Korea',
  '1,947,133',
  '2025',
  '1,712,793',
  '2023',
  '1,673,916',
  '2022'],
 [

In [106]:
# country , gdp, year
e.save_json(raws)

In [32]:
try:
    with sqlite3.connect('World_Economies.db') as conn:
        print(f"Opened SQLite database with version {sqlite3.sqlite_version} successfully.")

except sqlite3.OperationalError as e:
    print("Failed to open database:", e)
    
table.to_sql('Countries_by_GDP', conn)

Opened SQLite database with version 3.39.5 successfully.


72

In [87]:
import logging, logging.config
from functools import wraps

logger = logging.getLogger()
if not logger.handlers:
    # 사용자 정의 Formatter
    formatter = logging.Formatter(
        fmt='%(asctime)s, %(message)s',
        datefmt='%Y-%B-%d-%H-%M-%S'  # 원하는 날짜 포맷
    )

    # 핸들러 생성 및 설정
    handler = logging.FileHandler('etl_project_log.txt')
    handler.setFormatter(formatter)

    # 로거 설정
    logger.setLevel(logging.INFO)
    logger.addHandler(handler)
# # 기존 핸들러 제거
# for handler in logger.handlers[:]:
#     logger.removeHandler(handler)

# 테스트 로그 메시지
logger.info("This is a test log message.")

def logging_time(func):         
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = datetime.datetime.now()
        # formatted_start = start_time.strftime("%Y-%B-%d-%H-%M-%S")

        logging.info(f"Started '{func.__name__}'")
        
        result = func(*args, **kwargs)
        
        end_time = datetime.datetime.now()
        # formatted_end = end_time.strftime("%Y-%B-%d-%H-%M-%S")

        logging.info(f"Finished '{func.__name__}'")
        # logging.info(f"Execution time for '{func.__name__}': {end_time - start_time}")
        return result
    return wrapper

In [88]:
@logging_time
def sample_function(duration):
    """샘플 함수: 주어진 시간(초) 동안 대기"""
    import time
    time.sleep(duration)
    return "Done!"
result_sample = sample_function(2)
print(result_sample)

Done!


In [ ]:
Algeria
Angola
Benin
Botswana
Burkina Faso
Burundi
Cameroon
Cape Verde
Central African Republic
Chad
Comoros
Congo, Dem. Rep. of
Congo, Republic of
Côte d'Ivoire
Djibouti
Egypt
Equatorial Guinea
Eritrea
Eswatini
Ethiopia
Gabon
Gambia, The
Ghana
Guinea
Guinea-Bissau
Kenya
Lesotho
Liberia
Libya
Madagascar
Malawi
Mali
Mauritania
Mauritius
Mayotte
Morocco
Mozambique
Namibia
Niger
Nigeria
Reunion
Rwanda
Saint Helena
São Tomé and Príncipe
Senegal
Seychelles
Sierra Leone
Somalia
South Africa
South Sudan
Sudan
Tanzania
Togo
Tunisia
Uganda
Zambia
Zimbabwe

# 추가 요구 사항
## 코드를 수정해서 아래 요구사항을 구현하세요.
- 추출한 데이터를 데이터베이스에 저장하세요. 'Countries_by_GDP'라는 테이블명으로 'World_Economies.db'라는 데이터 베이스에 저장되어야 합니다. <br>해당 테이블은 'Country', 'GDP_USD_billion'라는 어트리뷰트를 반드시 가져야 합니다.

- 데이터베이스는 sqlite3 라이브러리를 사용해서 만드세요.

- 필요한 모든 작업을 수행하는 'etl_project_gdp_with_sql.py' 코드를 작성하세요.

## 화면 출력
- SQL Query를 사용해야 합니다.

- GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.

- 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.

### REGION Categorize

In [214]:
def flatten(arr:list):
    return[
        item
        for sublist in arr for item in (flatten(sublist) 
        if isinstance(sublist, list)else [sublist])
    ]

url = 'https://www.imf.org/external/datamapper/region.htm#sea'
response = requests.get(url)

if response.status_code == 200:
    region_html = response.text

def region_categorize(html: str):
    '''
    Region : [소속 국가] 형식의 딕셔너리를 반환합니다.
    Param: html: str
    return: region: dict
    '''
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.select('div.fancy > table > tr')

    category = [[td.get_text(strip=False) for td in tr.find_all('th')] for tr in table]
    for i in range(len(category) - 1, -1, -1):  # 인덱스를 뒤에서부터 순회
        if not category[i]: del category[i] # 리스트가 비어있으면 제거
        
    nara = [[td.get_text(strip=False) for td in tr.find_all('p')] for tr in table]
    for i in range(len(nara) - 1, -1, -1):  # 인덱스를 뒤에서부터 순회
        if not nara[i]: del nara[i] # 리스트가 비어있으면 제거
        
    region_arr = []
    for c in category:
        if len(c) == 0: continue
        region_arr.append(re.sub(r'[\r\n\xa0]', '', c[0]))
        
    for i, line in enumerate(nara):
        tmp = []
        for subline in line:
            tmp.append(subline.split('\n'))
        tmp = flatten(tmp)
        
        for j in range(len(tmp)-1, -1, -1):
            text = tmp[j]
            text = re.sub(r'\r', '', text)
            # '\xa0\xa0'은 ''으로 대체하고, '\xa0' 한 개는 ' '으로 대체
            text = re.sub(r'(?<=[A-Za-z])\xa0\xa0(?=[A-Za-z])', ' ', text)
            text = re.sub(r'\xa0\xa0', '', text) 
            text = re.sub(r'\xa0', ' ', text) 
            tmp[j] = text.strip()    
            if text == '': 
                del tmp[j]
                continue
        nara[i] = tmp
        
    region = {}
    for k, v in zip(region_arr, nara):
        region[k] = v
    
    return region
region = region_categorize(region_html)
region

{'Africa': ['Algeria',
  'Angola',
  'Benin',
  'Botswana',
  'Burkina Faso',
  'Burundi',
  'Cameroon',
  'Cape Verde',
  'Central African Republic',
  'Chad',
  'Comoros',
  'Congo, Dem. Rep. of',
  'Congo, Republic of',
  "Côte d'Ivoire",
  'Djibouti',
  'Egypt',
  'Equatorial Guinea',
  'Eritrea',
  'Eswatini',
  'Ethiopia',
  'Gabon',
  'Gambia, The',
  'Ghana',
  'Guinea',
  'Guinea-Bissau',
  'Kenya',
  'Lesotho',
  'Liberia',
  'Libya',
  'Madagascar',
  'Malawi',
  'Mali',
  'Mauritania',
  'Mauritius',
  'Mayotte',
  'Morocco',
  'Mozambique',
  'Namibia',
  'Niger',
  'Nigeria',
  'Reunion',
  'Rwanda',
  'Saint Helena',
  'São Tomé and Príncipe',
  'Senegal',
  'Seychelles',
  'Sierra Leone',
  'Somalia',
  'South Africa',
  'South Sudan',
  'Sudan',
  'Tanzania',
  'Togo',
  'Tunisia',
  'Uganda',
  'Zambia',
  'Zimbabwe'],
 'North Africa': ['Algeria', 'Egypt', 'Libya', 'Morocco', 'Tunisia'],
 'Sub-Saharan Africa': ['Angola',
  'Benin',
  'Botswana',
  'Burkina Faso',
  'B

In [215]:
region

{'Africa': ['Algeria',
  'Angola',
  'Benin',
  'Botswana',
  'Burkina Faso',
  'Burundi',
  'Cameroon',
  'Cape Verde',
  'Central African Republic',
  'Chad',
  'Comoros',
  'Congo, Dem. Rep. of',
  'Congo, Republic of',
  "Côte d'Ivoire",
  'Djibouti',
  'Egypt',
  'Equatorial Guinea',
  'Eritrea',
  'Eswatini',
  'Ethiopia',
  'Gabon',
  'Gambia, The',
  'Ghana',
  'Guinea',
  'Guinea-Bissau',
  'Kenya',
  'Lesotho',
  'Liberia',
  'Libya',
  'Madagascar',
  'Malawi',
  'Mali',
  'Mauritania',
  'Mauritius',
  'Mayotte',
  'Morocco',
  'Mozambique',
  'Namibia',
  'Niger',
  'Nigeria',
  'Reunion',
  'Rwanda',
  'Saint Helena',
  'São Tomé and Príncipe',
  'Senegal',
  'Seychelles',
  'Sierra Leone',
  'Somalia',
  'South Africa',
  'South Sudan',
  'Sudan',
  'Tanzania',
  'Togo',
  'Tunisia',
  'Uganda',
  'Zambia',
  'Zimbabwe'],
 'North Africa': ['Algeria', 'Egypt', 'Libya', 'Morocco', 'Tunisia'],
 'Sub-Saharan Africa': ['Angola',
  'Benin',
  'Botswana',
  'Burkina Faso',
  'B

In [216]:

import time
start = time.time()

def data_from_IMF():
    '''
    IMF의 API를 이용하여 데이터를 추출합니다.
    return: gdps: dict {country : GDP}
    '''
    year = datetime.datetime.now().strftime('%Y')
    nameURL = 'https://www.imf.org/external/datamapper/api/v1/countries'
    gdpURL = f'https://www.imf.org/external/datamapper/api/v1/NGDPD?periods={year}'

    response = requests.get(nameURL)
    names = response.text
    namedict = json.loads(names)['countries'] # 국가 코드 : 국가명 딕셔너리
    # 여기까지 2초 ~ 3초

    rspn = requests.get(gdpURL)
    gdptext = rspn.text
    gdpdict = json.loads(gdptext)['values']['NGDPD']
    # 여기까지 3초 ~ 5초

    gdps = {}
    for k, v in gdpdict.items():
        if k in namedict.keys(): # gdpdict에는 국가 이외의 대륙 분류 등이 섞여있음
            # if year in v.keys():
            gdps[namedict[k]['label']] = round(v[year], 2)
            # else:
                # gdps[namedict[k]['label']] = -1
    return gdps
gdps = data_from_IMF()
# print(f"{time.time()-start:.4f} sec")            

In [217]:
# print(type(datetime.datetime.now().strftime('%Y')))
gdps

{'Sudan': 29.99,
 'El Salvador': 37.84,
 'Estonia': 45.31,
 'Eswatini': 5.42,
 'Fiji': 6.08,
 'Finland': 319.99,
 'Gabon': 20.97,
 'Georgia': 35.91,
 'Greece': 265.17,
 'Guinea': 27.3,
 'Haiti': 30.57,
 'Honduras': 38.98,
 'Hungary': 245.62,
 'Iceland': 35.38,
 'Bulgaria': 115.53,
 'Burkina Faso': 23.56,
 'Cambodia': 51.16,
 'Central African Republic': 3.03,
 "China, People's Republic of": 19534.89,
 'Colombia': 419.33,
 'Congo, Dem. Rep. of the': 79.24,
 'Costa Rica': 100.67,
 'Croatia': 96.03,
 'Cyprus': 37.69,
 'Denmark': 431.23,
 'Dominican Republic': 135.54,
 'Brunei Darussalam': 16.68,
 'Equatorial Guinea': 12.92,
 'Italy': 2459.6,
 'Jamaica': 21.59,
 'Mexico': 1817.82,
 'Puerto Rico': 122.23,
 'Qatar': 226.22,
 'Switzerland': 999.6,
 'Andorra': 4.07,
 'Angola': 118.41,
 'Argentina': 574.2,
 'Armenia': 26.58,
 'Australia': 1881.14,
 'Austria': 559.22,
 'Bangladesh': 481.86,
 'Belgium': 689.36,
 'Bhutan': 3.52,
 'Bolivia': 51.3,
 'Botswana': 22.11,
 'Brazil': 2307.16,
 'Tunisia': 

In [218]:
i = 0
for k, v in gdps.items():
    if v >= 100:
        i += 1
i

72

In [219]:
# Step 1: region_to_country 데이터를 DataFrame으로 변환
region_list = []
for region, countries in region.items():
    for country in countries:
        region_list.append({"Region": region, "Country": country})
region_df = pd.DataFrame(region_list)
# Step 2: country_to_gdp 딕셔너리를 DataFrame으로 변환
gdp_df = pd.DataFrame(list(gdps.items()), columns=["Country", "GDP"])

# Step 3: 두 DataFrame 병합 (Country를 기준으로)
merged_df = pd.merge(region_df, gdp_df, on="Country", how="inner")

# Step 4: 정리된 결과 출력
print(merged_df)

# 추가: Region 별 국가 및 GDP 보기
grouped = merged_df.groupby("Region")[["Country", "GDP"]].apply(lambda x: x.values.tolist())
grouped

            Region       Country     GDP
0           Africa       Algeria  264.27
1           Africa        Angola  118.41
2           Africa         Benin   23.07
3           Africa      Botswana   22.11
4           Africa  Burkina Faso   23.56
..             ...           ...     ...
341  South America      Colombia  419.33
342  South America       Ecuador  125.66
343  South America      Paraguay   46.80
344  South America          Peru  294.90
345  South America       Uruguay   86.42

[346 rows x 3 columns]


Region
Africa                           [[Algeria, 264.27], [Angola, 118.41], [Benin, ...
Asia and Pacific                 [[Armenia, 26.58], [Australia, 1881.14], [Azer...
Australia and New Zealand            [[Australia, 1881.14], [New Zealand, 262.92]]
Caribbean                        [[Antigua and Barbuda, 2.44], [Aruba, 4.45], [...
Central America                  [[Costa Rica, 100.67], [Dominican Republic, 13...
Central Asia and the Caucasus    [[Armenia, 26.58], [Azerbaijan, 77.04], [Georg...
East Asia                        [[China, People's Republic of, 19534.89], [Hon...
Eastern Europe                   [[Albania, 27.99], [Belarus, 76.87], [Bosnia a...
Europe                           [[Albania, 27.99], [Andorra, 4.07], [Austria, ...
Middle East                      [[Bahrain, 49.53], [Iraq, 270.87], [Israel, 55...
North Africa                     [[Algeria, 264.27], [Egypt, 345.87], [Libya, 4...
North America                    [[Canada, 2330.31], [Mexico, 1817.82], [United.

In [220]:
merged_df.GDP = merged_df.GDP.astype(int)
top_5_by_region = (
    merged_df.sort_values(by=['Region', 'GDP'], ascending=[True, False])
    .groupby('Region')
    .head(5)
)
region_avg_gdp = top_5_by_region.groupby("Region")["GDP"].mean().reset_index().apply(lambda x: round(x, 2))
region_avg_gdp.columns = ["Region", "@5_GDP_Avg"]

In [221]:
region_avg_gdp

,Region,@5_GDP_Avg
0,Africa,277.80
1,Asia and Pacific,6404.40
2,Australia and New Zealand,1071.50
3,Caribbean,67.40
4,Central America,96.80
5,Central Asia and the Caucasus,127.20
6,East Asia,5421.20
7,Eastern Europe,824.20
8,Europe,3317.60
9,Middle East,550.00


In [222]:
conn.execute("DROP TABLE Region_Category")
try:
    with sqlite3.connect('World_Economies.db') as conn:
        print(f"Opened SQLite database with version {sqlite3.sqlite_version} successfully.")

except sqlite3.OperationalError as e:
    print("Failed to open database:", e)
    
region_df.to_sql('Region_Category', conn)
region_df

Opened SQLite database with version 3.39.5 successfully.


,Region,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina Faso
...,...,...
452,South America,French Guiana
453,South America,Paraguay
454,South America,Peru
455,South America,Uruguay


In [193]:
asdf = '\xa0\xa0ABC\xa0D' 
text = re.sub(r'\xa0\xa0', '', asdf) 
text = re.sub(r'\xa0', ' ', text) 
text

'ABC D'